https://www.blog.pythonlibrary.org/2010/02/03/another-step-by-step-sqlalchemy-tutorial-part-1-of-2/

View db with DB browser for SQLite http://sqlitebrowser.org/

Cheat sheet https://www.pythonsheets.com/notes/python-sqlalchemy.html

In [2]:
import sqlalchemy
print(sqlalchemy.__version__)

1.1.13


In [30]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Column, Table, ForeignKey
from sqlalchemy import Integer, String
from sqlalchemy import inspect
from sqlalchemy.sql import select
from sqlalchemy.sql import and_
import pandas as pd

In [7]:
engine = create_engine('sqlite:///tutorial.db',
                       echo=True)

Next step creates the .db file

In [8]:
metadata = MetaData(bind=engine)
 
users_table = Table('users', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('name', String(40)),
                    Column('age', Integer),
                    Column('password', String),
                    )
 
addresses_table = Table('addresses', metadata,
                        Column('id', Integer, primary_key=True),
                        Column('user_id', None, ForeignKey('users.id')),
                        Column('email_address', String, nullable=False)                            
                        )
 
# create tables in database
metadata.create_all()

2017-12-23 15:24:09,136 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-12-23 15:24:09,137 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 15:24:09,140 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-12-23 15:24:09,142 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 15:24:09,146 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-12-23 15:24:09,149 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 15:24:09,154 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2017-12-23 15:24:09,157 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 15:24:09,160 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(40), 
	age INTEGER, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-12-23 15:24:09,163 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 15:24:09,167 INFO sqlalchemy.engine.base.Engine COMMIT
2017-12-23 15:24:09,170 INFO 

In [26]:
inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())

2017-12-23 15:39:52,803 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2017-12-23 15:39:52,805 INFO sqlalchemy.engine.base.Engine ()
['addresses', 'users']


In [29]:
# Get column information
print(inspector.get_columns('users'))

[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'name', 'type': VARCHAR(length=40), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'age', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'password', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


### Insert some data

In [10]:
# create an Insert object
ins = users_table.insert()
# add values to the Insert object
new_user = ins.values(name="Joe", age=20, password="pass")
 
# create a database connection
conn = engine.connect()
# add user to database by executing SQL
conn.execute(new_user)

2017-12-23 15:27:52,205 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age, password) VALUES (?, ?, ?)
2017-12-23 15:27:52,208 INFO sqlalchemy.engine.base.Engine ('Joe', 20, 'pass')
2017-12-23 15:27:52,213 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
# a connectionless way to Insert a user
ins = users_table.insert()
result = engine.execute(ins, name="Shinji", age=15, password="nihongo")
 
# another connectionless Insert
result = users_table.insert().execute(name="Martha", age=45, password="dingbat")

2017-12-23 15:28:59,321 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age, password) VALUES (?, ?, ?)
2017-12-23 15:28:59,324 INFO sqlalchemy.engine.base.Engine ('Shinji', 15, 'nihongo')
2017-12-23 15:28:59,327 INFO sqlalchemy.engine.base.Engine COMMIT
2017-12-23 15:28:59,331 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age, password) VALUES (?, ?, ?)
2017-12-23 15:28:59,334 INFO sqlalchemy.engine.base.Engine ('Martha', 45, 'dingbat')
2017-12-23 15:28:59,337 INFO sqlalchemy.engine.base.Engine COMMIT


Insert multiple rows

In [12]:
conn.execute(users_table.insert(), [
    {"name": "Ted", "age":10, "password":"dink"},
    {"name": "Asahina", "age":25, "password":"nippon"},
    {"name": "Evan", "age":40, "password":"macaca"}
])

2017-12-23 15:29:31,756 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age, password) VALUES (?, ?, ?)
2017-12-23 15:29:31,760 INFO sqlalchemy.engine.base.Engine (('Ted', 10, 'dink'), ('Asahina', 25, 'nippon'), ('Evan', 40, 'macaca'))
2017-12-23 15:29:31,763 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
### Selecting
 
s = select([users_table])
result = s.execute()
 
for row in result:
    print(row)

2017-12-23 15:31:55,820 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.age, users.password 
FROM users
2017-12-23 15:31:55,823 INFO sqlalchemy.engine.base.Engine ()
(1, 'Joe', 20, 'pass')
(2, 'Shinji', 15, 'nihongo')
(3, 'Martha', 45, 'dingbat')
(4, 'Ted', 10, 'dink')
(5, 'Asahina', 25, 'nippon')
(6, 'Evan', 40, 'macaca')


Get all the results in a list of tuples

In [15]:
conn = engine.connect()
res = conn.execute(s)
rows = res.fetchall()

2017-12-23 15:32:52,322 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.age, users.password 
FROM users
2017-12-23 15:32:52,326 INFO sqlalchemy.engine.base.Engine ()


In [16]:
rows

[(1, 'Joe', 20, 'pass'),
 (2, 'Shinji', 15, 'nihongo'),
 (3, 'Martha', 45, 'dingbat'),
 (4, 'Ted', 10, 'dink'),
 (5, 'Asahina', 25, 'nippon'),
 (6, 'Evan', 40, 'macaca')]

In [18]:
s = select([users_table.c.name, users_table.c.age])
result = conn.execute(s)
for row in result:
    print(row)

2017-12-23 15:33:53,578 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.age 
FROM users
2017-12-23 15:33:53,580 INFO sqlalchemy.engine.base.Engine ()
('Joe', 20)
('Shinji', 15)
('Martha', 45)
('Ted', 10)
('Asahina', 25)
('Evan', 40)


In [20]:
# The following is the equivalent to 
# SELECT * FROM users WHERE id > 3
s = select([users_table], users_table.c.id > 3)
result = conn.execute(s)
for row in result:
    print(row)

2017-12-23 15:35:08,229 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.age, users.password 
FROM users 
WHERE users.id > ?
2017-12-23 15:35:08,232 INFO sqlalchemy.engine.base.Engine (3,)
(4, 'Ted', 10, 'dink')
(5, 'Asahina', 25, 'nippon')
(6, 'Evan', 40, 'macaca')


In [23]:
# You can use the "and_" module to AND multiple fields together

s = select([users_table]).where(and_(users_table.c.name == 'Martha',users_table.c.age < 25))

result = conn.execute(s)
for row in result:
    print(row)

2017-12-23 15:37:31,234 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.age, users.password 
FROM users 
WHERE users.name = ? AND users.age < ?
2017-12-23 15:37:31,237 INFO sqlalchemy.engine.base.Engine ('Martha', 25)


## Pandas
Lets get the data as a dataframe

In [63]:
sql = "select * from users"
print('Performing query...')
df = pd.read_sql(sql, con=engine)

Performing query...
2017-12-23 15:59:15,676 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("select * from users")
2017-12-23 15:59:15,679 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 15:59:15,682 INFO sqlalchemy.engine.base.Engine select * from users
2017-12-23 15:59:15,684 INFO sqlalchemy.engine.base.Engine ()


In [64]:
df

,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca


In [72]:
df_copy = df.copy()

In [73]:
df_copy

,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca


In [74]:
new_entry = {}
new_entry['id'] = 6
new_entry['name'] = 'Robin'
new_entry['age'] = 100
new_entry['password'] = 'foo'
new_entry

{'age': 100, 'id': 6, 'name': 'Robin', 'password': 'foo'}

In [83]:
new_entry_s = pd.Series(new_entry) # Create the series from a dic

In [84]:
new_entry_s

age           100
id              6
name        Robin
password      foo
dtype: object

In [89]:
df_copy = df_copy.append(new_entry_s, ignore_index=True)

In [90]:
df_copy

,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca
6,6,Robin,100,foo


In [93]:
df_copy.to_sql(name='users', con=engine, if_exists='replace', index=False)

2017-12-23 16:10:02,321 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-12-23 16:10:02,325 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 16:10:02,328 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-12-23 16:10:02,330 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 16:10:02,333 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2017-12-23 16:10:02,336 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 16:10:02,339 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-12-23 16:10:02,342 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 16:10:02,346 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'users' AND type = 'table'
2017-12-23 16:10:02,349 INFO sqlalchemy.engine.base.Engine ()
2017-12-23 16:10:02,352 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("users")
2017-12-23 16:10:02,354 IN

In [94]:
pd.read_sql_query("select * from users;", conn)

2017-12-23 16:10:04,339 INFO sqlalchemy.engine.base.Engine select * from users;
2017-12-23 16:10:04,344 INFO sqlalchemy.engine.base.Engine ()


,id,name,age,password
0,1,Joe,20,pass
1,2,Shinji,15,nihongo
2,3,Martha,45,dingbat
3,4,Ted,10,dink
4,5,Asahina,25,nippon
5,6,Evan,40,macaca
6,6,Robin,100,foo
